# SENG 550 Final Project 

Luka Petrovic UCID: 30077345
Logan Boras UCID: <br> 
Kenny Jeon UCID: 30068677

### 1. Set up environment and Spark ###

In [2]:
!python3 -m pip install --upgrade pip
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# uncomment the following if you do not have spark installed in your project
#!wget --no-check-certificate https://dlcdn.apache.org/spark/spark-3.3.3/spark-3.3.3-bin-hadoop3.tgz
#!tar -xvf spark-3.3.3-bin-hadoop3.tgz
!pip install findspark
!pip install numpy
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/training/spark-3.3.3-bin-hadoop3"

In [3]:
import findspark
findspark.init()
import pyspark

sc = pyspark.SparkContext(appName="Seng550Project")

### 2. Create directory for input data to live ###

Note that the deerfoot.csv file or whatever input data you are using must be within the data folder then proceed to store it on the hdfs

In [4]:
!mkdir data
!hadoop fs -mkdir -p /inputs
!hadoop fs -put data /inputs

mkdir: cannot create directory ‘data’: File exists
put: `/inputs/data/deerfoot5Lines.csv': File exists
put: `/inputs/data/deerfoot.csv': File exists


### 3. Grab the specific data we want from the data set ###

In [7]:
file = '/inputs/data/deerfoot.csv'

deerfootRDD = (sc.textFile(file, 8))
'\n'.join(deerfootRDD.zipWithIndex().map(lambda x: str(x[1]) + ': ' + str(x[0])))

TypeError: can only join an iterable

#### 3a. Now we want to get the data only for morning and afternoon rush hour, 8am and 4pm will suffice ####

In [8]:
def getRushHours(RDD):

    rushHours = RDD.split(',')
    return (rushHours[1],rushHours[6],rushHours[13])

rushHoursRDD = deerfootRDD.map(getRushHours)
print(rushHoursRDD.take(1))

[('Saturday', '35', '35')]


#### 3b. Rush hour generally doesn't happen on weekends, so we need to filter out Saturdays and Sundays ####

In [9]:
# This function was taken from the Spark Basics Tutorial

def removeWeekends(deerfootRDDRecord):
    if deerfootRDDRecord[0]=="Saturday" or deerfootRDDRecord[0]=="Sunday":
        return False
    else:
        return True

weekdayRushHoursRDD = rushHoursRDD.filter(removeWeekends)
print(weekdayRushHoursRDD.take(5))

[('Monday', '45', '40'), ('Tuesday', '52', '40'), ('Wednesday', '39', '40'), ('Thursday', '49', '56'), ('Friday', '36', '43')]


### 4. Compute Statistics and Averages

For our analysis, we want to get both the weekly AM and PM averages, as well as the overall average. For a day to be considered "bad", the mean of the AM and PM commute times should be higher than the overall average commute time.

#### 4a. Get day counts

In [10]:
totalCount = weekdayRushHoursRDD.count()
print(totalCount)

# The below might be useful later, uncomment if we need counts of each day in dataset
# perDayRDD = weekdayRushHoursRDD.map(lambda x: (x[0],1))
# perDayCountsRDD = perDayRDD.reduceByKey(lambda x,y:x+y)

# dayCountsList = perDayCountsRDD.collect()
# dayCountsDict = dict(deerfootDayCountsList)

144


#### 4b. Get overall rush hour average

In [11]:
#testRdd = sc.parallelize([('Monday', '45', '40'), ('Tuesday', '52', '40'), ('Wednesday', '39', '40')])
totalRushHourSum = sc.accumulator(0) # total average


def sum(item):
    global totalRushHourSum
    totalRushHourSum += int(item[1])
    totalRushHourSum += int(item[2])
    
weekdayRushHoursRDD.foreach(sum)
totalRushHourAverage = totalRushHourSum.value/(totalCount*2)
print(totalRushHourAverage)    


41.795138888888886


#### Reduce down am and pm commute times into a daily average

In [14]:
weekdayAverageRushHoursRDD = weekdayRushHoursRDD.map(lambda x: (x[0], (int(x[1])+int(x[2]))/2))
print(weekdayAverageRushHoursRDD.take(5))
                                                    

[('Monday', 42.5), ('Tuesday', 46.0), ('Wednesday', 39.5), ('Thursday', 52.5), ('Friday', 39.5)]


### 5 - Prepare data for model

We want to add a "good" or "bad" label to each field. In order to do this, we will compare the daily average commute time with the total average. If it's higher, its a bad day. If its lower, consider it a good day. We will use the integer 1 as good and 0 as bad.

#### 5a - Add labels to data

We want to add a "good" or "bad" label to each field. In order to do this, we will compare the daily average commute time with the total average. If it's higher, its a bad day. If its lower, consider it a good day. We will use the integer 1 as good and 0 as bad.

In [17]:
def addLabels(item):
    goodOrBadLabel = 1
    if(item[1] > totalRushHourAverage):
        goodOrBadLabel = 0
        
    return (goodOrBadLabel,item[0], item[1])


labeledRDD = weekdayAverageRushHoursRDD.map(addLabels)
print(labeledRDD.take(2))

[(0, 'Monday', 42.5), (0, 'Tuesday', 46.0)]


#### 5b - Convert days of the week to integers
We want to use the weekday as a feature, but you can't use a string as a feature. This will convert the weekday into a 1-5 value, 1 being monday and 5 being friday.

In [20]:
def stringToInt(item):
    weekdayInt = 0;
    weekday = item[1]
    
    if weekday == 'Monday':
        weekdayInt = 1
    elif weekday == 'Tuesday':
        weekdayInt = 2
    elif weekday == 'Wednesday':
        weekdayInt = 3
    elif weekday == 'Thursday':
        weekdayInt = 4
    elif weekday == 'Friday':
        weekdayInt = 5
    else:
        weekdayInt = 1  # Bad data, default to Monday
      
    return (item[0], weekdayInt, item[2])    
        
labeledConvertedRDD = labeledRDD.map(stringToInt)
print(labeledConvertedRDD.take(2))

[(0, 1, 42.5), (0, 2, 46.0)]


#### 5c - Convert data to LabeledPoints

In [27]:
from pyspark.mllib.regression import LabeledPoint

def parsePoint(item):
    labeledPoint = LabeledPoint(item[0], item[1:])
    return labeledPoint

finalRDD = labeledConvertedRDD.map(parsePoint)
print(finalRDD.take(1))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2230)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2259)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2278)
	at org.apache.spark.api.python.PythonRDD$.runJob(PythonRDD.scala:166)
	at org.apache.spark.api.python.PythonRDD.runJob(PythonRDD.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.inv

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 52926)
Traceback (most recent call last):
  File "/opt/bitnami/python/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/bitnami/python/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/bitnami/python/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/bitnami/python/lib/python3.8/socketserver.py", line 720, in __init__
    self.handle()
  File "/training/spark-3.3.3-bin-hadoop3/python/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/training/spark-3.3.3-bin-hadoop3/python/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/training/spark-3.3.3-bin-hadoop3/python/pyspark/accumulators.py", line 257, in ac

### 6. Set up logistic regression model for prediction

Now that we have our inputs sorted, we can start to make our model

#### 6a - Make and train a model

In [26]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel

trainingData, testingData = finalRDD.randomSplit([.8, .2],21)
print(trainingData.count())
print(testingData.count())

trainingData.cache()
testingData.cache()

model = LogisticRegressionWithLBFGS.train(finalRDD)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2230)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2259)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2278)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2303)
	at org.apache.spark.rdd.RDD.$anonfun$collect$1(RDD.scala:1021)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:406)
	at org.apache.spark.rdd.RDD.collect(RDD.scala:1020)
	at org.apache.spark.api.python.PythonRDD$.collectAndServe(PythonRDD.scala:180)
	at org.apache.spark.api.python.PythonRDD.collectAndServe(PythonRDD.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.re

### 7. Evalute Model

Now that we have our model built and trained, we will evaluate it with test data

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/training/spark-3.3.3-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/training/spark-3.3.3-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/training/spark-3.3.3-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
/training/spark-3.3.3-bin-hadoop3/python/pyspark/context.py:561: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been k